# ETL SINIESTROS VIALES 
# Ciudad Autonoma de Buenos Aires CABA

**Por** 

**Data Analyst**

**GABRIEL GOMEZ RESTREPO**





Para  los Datasets  este  estudio sobre los accidentes viales de la Ciudad Autonoma de Buenos Aires(CABA), Argentina.
recurrimos a consultar  la siguientes links: 

**Para siniestros**

homicidios.xlsx   ( Hechos y Victimas)

https://data.buenosaires.gob.ar/dataset/victimas-siniestros-viales

**Para poblacion por Comuna** 

PBP_CO1025.xls   (poblacion por comuna  por edad y sexo)

proy_1025_depto_caba.xls   (resumen de PBP_CO1025, poblacion por comuna y por sexo)

https://data.buenosaires.gob.ar/dataset/estructura-poblacion


http://www.indec.mecon.ar/ftp/cuadros/poblacion

**Para Bariios por Comuna** 

comunas.xlsx  (Barrios por comuna)

https://data.buenosaires.gob.ar/dataset/comunas

La revision inicial de los datos que hemos otenido requiere de su estraccion tranformacion y  finalmente Cargar o crear una nueva base de datos usando nombres auque similares para  diferenciarlos con las fuentes originales.
## Librerias 
Las siguientes librerias son las que utilice. 

* **Pandas (import pandas as pd):**
Pandas es una biblioteca de Python fundamental en manejo de DataFrame que proporciona estructuras de datos flexibles y herramientas de análisis. 

* **NumPy (import numpy as np):**
NumPy es una biblioteca fundamental en Python para realizar operaciones numéricas.

* **%load_ext autoreload y %autoreload 2:**
Estos comandos son específicos de los cuadernos Jupyter y se utilizan para recargar automáticamente módulos antes de ejecutar una celda. %load_ext autoreload habilita la recarga automática de módulos, y %autoreload 2 configura la recarga automática para que sea más agresiva y recargue incluso las funciones de los módulos.

* **Warnings (import warnings):**
El módulo warnings proporciona herramientas para controlar las advertencias que emite Python. En este caso, se está configurando para ignorar las advertencias, lo cual puede ser útil para evitar que las advertencias llenen la salida de la consola y distraigan durante la ejecución del código.

## Funciones 

* **Las funciones nos ahorran tiempo:**
>Esta son las funciones que utilice, generar unas funciones para agilizar nuestras tareas es muy importante evitando muchas acciones repetitivas ademas permiten un proceso mas claro y ordenado a continuacion las relaciono:  

ver_nulos, ver_duplicados, ver_tipo_datos, conver_tiempo, imputa_frecuente, imputa_edad_sexo

# DATAFRAME df_hechos y df_victimas

**La idea iniocial es crear 2 Dataframe ('df_hechos' y df_victimas )  empercemos con :** 

## df_hehos

In [1]:
import numpy as np
import pandas as pd

In [2]:
df_hechos = pd.read_excel('./Datasets/Fuente/homicidios.xlsx', sheet_name='HECHOS')

In [3]:
df_hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   696 non-null    object        
 7   HH                     696 non-null    object        
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Altura                 129 non-null    float64       
 12  Cruce                  525 non-null    object        
 13  Direc

**Emecemos cambiando los titulos de las columnas para mas fasilidad de entendimiento, ademmas usaremos la primera letra en mayusculas y las demas en minuscula, para mejor presentacion**

In [8]:
df_hechos.columns = df_hechos.columns.str.replace('_', ' ') # guiones por espacios
df_hechos.columns = [x.capitalize() for x in df_hechos.columns] #  mayúscula letra 1 y resto a minusculas
# Cambiamos algunos titulos 
df_hechos = df_hechos.rename(columns={'N victimas': 'Cantidad víctimas', 'Aaaa':'Año', 'Mm':'Mes', 'Dd':'Día',
                                      'Hh':'Hora entera', 'Xy (caba)':'XY (CABA)', 'Victima': 'Víctima'})

df_hechos.columns # oservando como quedan los titulares de las columnas

Index(['Id', 'Cantidad víctimas', 'Fecha', 'Año', 'Mes', 'Día', 'Hora',
       'Hora entera', 'Lugar del hecho', 'Tipo de calle', 'Calle', 'Altura',
       'Cruce', 'Dirección normalizada', 'Comuna', 'XY (CABA)', 'Pos x',
       'Pos y', 'Participantes', 'Víctima', 'Acusado'],
      dtype='object')

## Nulos
**Usuamos Funcion 'ver_nulos' para revisar ausencias de datos aunque ya df_hechos.info() 

In [9]:
def ver_nulos(df):
    # Creamos Diccionario para almacenar información que queremos ver
    dic = {"campo": [], "tipoDato": [], "%NOnull": [], "%null": [], "nulos": []}      
    for columna in df.columns:  # Iterar sobre todos los campo del DataFrame df
        # Calcular el porcentaje de No nulos en la columna actual
        porcentaje_no_nulos = (df[columna].count() / len(df)) * 100
        # LLenamos  el Diccionario
        dic["campo"].append(columna)
        dic["tipoDato"].append(df[columna].apply(type).unique())
        dic["%NOnull"].append(round(porcentaje_no_nulos, 2))
        dic["%null"].append(round(100 - porcentaje_no_nulos, 2))
        dic["nulos"].append(df[columna].isnull().sum())             
    df_info = pd.DataFrame(dic) # Crearmos otro DataFrame con la información nueva 
    nulos = df.isnull().sum().sum() # Calcular los  nulos en el DataFrame      
    print(f'TOTAL NULOS  DataFrame: {nulos}')  # Imprimir el total de valores nulos
        # Mostrar los campos o columnas  con valores nulos y % 
    campo_con_nulos = df_info[df_info["nulos"] > 0][["campo", "%null"]].sort_values(by="%null", ascending=False)
    if not campo_con_nulos.empty:
        print("\nCAMPOS QUE TIENEN VALORES NULOS Y SU  % :")
        print(campo_con_nulos)    
    return df_info  # Retonamos el DataFrame con la información de cada columna


In [10]:
ver_nulos(df_hechos)

TOTAL NULOS  DataFrame: 747

CAMPOS QUE TIENEN VALORES NULOS Y SU  % :
                    campo  %null
11                 Altura  81.47
12                  Cruce  24.57
13  Dirección normalizada   1.15
10                  Calle   0.14


,campo,tipoDato,%NOnull,%null,nulos
0,Id,[<class 'str'>],100.00,0.00,0
1,Cantidad víctimas,[<class 'int'>],100.00,0.00,0
2,Fecha,[<class 'pandas._libs.tslibs.timestamps.Timest...,100.00,0.00,0
3,Año,[<class 'int'>],100.00,0.00,0
4,Mes,[<class 'int'>],100.00,0.00,0
5,Día,[<class 'int'>],100.00,0.00,0
6,Hora,"[<class 'datetime.time'>, <class 'str'>, <clas...",100.00,0.00,0
7,Hora entera,"[<class 'int'>, <class 'str'>]",100.00,0.00,0
8,Lugar del hecho,[<class 'str'>],100.00,0.00,0
9,Tipo de calle,[<class 'str'>],100.00,0.00,0


In [17]:
ver_nulos(df_hechos[['Calle', 'Altura',
       'Cruce', 'Dirección normalizada']])

TOTAL NULOS  DataFrame: 747

CAMPOS QUE TIENEN VALORES NULOS Y SU  % :
                   campo  %null
1                 Altura  81.47
2                  Cruce  24.57
3  Dirección normalizada   1.15
0                  Calle   0.14


,campo,tipoDato,%NOnull,%null,nulos
0,Calle,"[<class 'str'>, <class 'float'>]",99.86,0.14,1
1,Altura,[<class 'float'>],18.53,81.47,567
2,Cruce,"[<class 'str'>, <class 'float'>]",75.43,24.57,171
3,Dirección normalizada,"[<class 'str'>, <class 'float'>]",98.85,1.15,8


In [18]:
df_hechos[['Calle', 'Altura',
       'Cruce', 'Dirección normalizada']]

,Calle,Altura,Cruce,Dirección normalizada
0,PIEDRA BUENA AV.,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G..."
1,"PAZ, GRAL. AV.",NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV."
2,ENTRE RIOS AV.,2034.0,NaN,ENTRE RIOS AV. 2034
3,LARRAZABAL AV.,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL."
4,SAN JUAN AV.,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES."
...,...,...,...,...
691,RIESTRA AV.,NaN,MOM,RIESTRA AV. y MOM
692,"DELLEPIANE, LUIS, TTE. GRAL.",NaN,LACARRA AV.,"DELLEPIANE, LUIS, TTE. GRAL. y LACARRA AV."
693,GAONA AV.,NaN,TERRADA,GAONA AV. y TERRADA
694,"PERON, EVA AV.",4071.0,NaN,"PERON, EVA AV. 4071"


## Eliminar Columna Altura 

Vamos a eliminar la columna 'Altura' ya tiene un porcentage muy alto de nulos, y  su informacion la podemos conseguir en 

las columnas 'Direccion normalizada' y en 'Lugar del hecho'